In [1]:
from transformers import BertTokenizer, TFBertModel,BertForQuestionAnswering,BertForMaskedLM
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline
import logging
import tensorflow as tf
import torch

# Set the logging level to WARNING (or ERROR) to suppress the warning
logging.getLogger("transformers.modeling_utils").setLevel(logging.WARNING)

In [2]:
tokenizer = BertTokenizer.from_pretrained('./pretrained')
QA = BertForQuestionAnswering.from_pretrained("./QA_model")
typo_fix = T5ForConditionalGeneration.from_pretrained("./typo")
typo_tokenizer = T5Tokenizer.from_pretrained("./typo_tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
typo_tokenizer.save_pretrained("./typo_tokenizer")

('./typo_tokenizer\\tokenizer_config.json',
 './typo_tokenizer\\special_tokens_map.json',
 './typo_tokenizer\\spiece.model',
 './typo_tokenizer\\added_tokens.json')

In [ ]:
sentence = "Ths is an exmple sentence with typos"
input_ids = tokenizer.encode(sentence, return_tensors='pt')
mask_idx = input_ids[0].tolist().index(tokenizer.mask_token_id)
input_ids[0, mask_idx] = tokenizer.mask_token_id
outputs = typo_fix(input_ids)
predictions = outputs[0][0, mask_idx].topk(5)
input_ids[0, mask_idx] = predictions.indices[0].item()
fixed_sent = tokenizer.decode(input_ids[0])
print(fixed_sent)

In [19]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

def typo_correction(input_sentence):

    input_sequence = f"fix typo: {input_sentence}"
    input_ids = typo_tokenizer(input_sequence, return_tensors="pt").input_ids

    # Generate the corrected sequence
    generated_ids = typo_fix.generate(input_ids)

    # Decode the generated sequence
    corrected_sentence = typo_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return corrected_sentence

sentence = "Ths is an exmple sentence with typos"
input_ids = typo_tokenizer(sentence,return_tensors = "pt")["input_ids"]
generated_ids = typo_fix.generate(input_ids)
gen_ids

# fixed_sentence = typo_correction(sentence)
# print("Original sentence:", sentence)
# print("Corrected sentence:", fixed_sentence)

NameError: name 'gen_ids' is not defined

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

def typo_correction(input_sentence, mask_idx, model):
    tokenizer = BertTokenizer.from_pretrained('./pretrained')
    
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt')
    input_ids[0, mask_idx] = tokenizer.mask_token_id
    
    outputs = model(input_ids)
    predictions = outputs[0][0, mask_idx].topk(5)
    input_ids[0, mask_idx] = predictions.indices[0].item()
    
    fixed_sent = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return fixed_sent

# Example usage
sentence = "Ths is an exmple sentence with typos"
mask_idx = 6  # Adjust based on the position of the typo
fixed_sentence = typo_correction(sentence, mask_idx, typo_fix)
print("Original sentence:", sentence)
print("Corrected sentence:", fixed_sentence)

In [8]:
question = "Who is te daughter of A?"
answer ="""B'S Father's C and her mother is A"""
inputs = tokenizer(question, answer, return_tensors="pt")

outputs = QA(**inputs)
start_scores = outputs.start_logits
end_scores = outputs.end_logits

start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)

answer_tokens = inputs["input_ids"][0][start_index:end_index + 1]
answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
answer

'b'